In [1]:
import tensorflow as tf
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.models import Sequential,Model
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.metrics import Accuracy
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint,ReduceLROnPlateau,EarlyStopping

In [2]:
image_height = 100
image_width = 100
batch_size = 32
epochs = 10
num_classes = 2  # Healthy and Parkinson classes

In [3]:
# Load pre-trained VGG16 model
base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(image_height, image_width, 3))


In [4]:
# Create a new model
# model = Sequential([
#     base_model,
#     Flatten(),
#     Dense(256, activation='relu'),
#     Dropout(0.5),
#     Dense(num_classes, activation='softmax')
# ])
hmodel = base_model.output
hmodel = Flatten()(hmodel)
hmodel = Dense(256,activation='relu')(hmodel)
hmodel = Dropout(0.5)(hmodel)
hmodel = Dense(num_classes,activation='softmax')(hmodel)

model = Model(inputs=base_model.input,outputs=hmodel)
# Freeze the pre-trained layers
for layer in base_model.layers:
    layer.trainable = False

In [5]:
base_model.summary()

Model: "inception_v3"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 100, 100, 3)]        0         []                            
                                                                                                  
 conv2d (Conv2D)             (None, 49, 49, 32)           864       ['input_1[0][0]']             
                                                                                                  
 batch_normalization (Batch  (None, 49, 49, 32)           96        ['conv2d[0][0]']              
 Normalization)                                                                                   
                                                                                                  
 activation (Activation)     (None, 49, 49, 32)           0         ['batch_normalizati

In [6]:
checkpoint = ModelCheckpoint(r'E:\project\PDD\srp',monitor='val_loss',save_best_only=True,verbose=3)

earlystop = EarlyStopping(monitor='val_loss',patience=7,verbose=3,restore_best_weights=True)
learning_rate = ReduceLROnPlateau(monitor='val_loss',patience=3,verbose=3)
callbacks=[checkpoint,earlystop,learning_rate]

In [7]:
# Compile the model
model.compile(optimizer='Adam',loss='binary_crossentropy',metrics=['accuracy'])

In [8]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2
)

In [9]:
test_datagen = ImageDataGenerator(rescale=1./255)


In [10]:
# Load training and validation data
train_generator = train_datagen.flow_from_directory(
    'datasets/Spiral/train',
    target_size=(image_height, image_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='training'
)
validation_generator = train_datagen.flow_from_directory(
    'datasets/Spiral/train',
    target_size=(image_height, image_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'
)

Found 1046 images belonging to 2 classes.
Found 260 images belonging to 2 classes.


In [11]:
test_generator = test_datagen.flow_from_directory(
    'datasets/Spiral/test',
    target_size=(image_height, image_width),
    batch_size=batch_size,
    class_mode='categorical'
)

Found 326 images belonging to 2 classes.


In [12]:
model.fit_generator(
    train_generator,
    steps_per_epoch=train_generator.samples//batch_size,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples//batch_size,
    callbacks=callbacks,
    epochs=15  # Ensure there are no hidden or non-printable characters in this line
)


C:\Users\Mukesh Raja\AppData\Local\Temp\ipykernel_1028\1332473610.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(


Epoch 1/15


32/32 [==============================] - ETA: 0s - loss: 0.6567 - accuracy: 0.6805
Epoch 1: val_loss improved from inf to 0.45679, saving model to E:\project\PDD\srp
INFO:tensorflow:Assets written to: E:\project\PDD\srp\assets


INFO:tensorflow:Assets written to: E:\project\PDD\srp\assets


32/32 [==============================] - 61s 2s/step - loss: 0.6567 - accuracy: 0.6805 - val_loss: 0.4568 - val_accuracy: 0.7852 - lr: 0.0010
Epoch 2/15
32/32 [==============================] - ETA: 0s - loss: 0.4889 - accuracy: 0.7623
Epoch 2: val_loss did not improve from 0.45679
32/32 [==============================] - 13s 373ms/step - loss: 0.4889 - accuracy: 0.7623 - val_loss: 0.4610 - val_accuracy: 0.7656 - lr: 0.0010
Epoch 3/15
32/32 [==============================] - ETA: 0s - loss: 0.5003 - accuracy: 0.7515
Epoch 3: val_loss improved from 0.45679 to 0.42400, saving model to E:\project\PDD\srp
INFO:tensorflow:Assets written to: E:\project\PDD\srp\assets


INFO:tensorflow:Assets written to: E:\project\PDD\srp\assets


...

32/32 [==============================] - 57s 2s/step - loss: 0.4340 - accuracy: 0.7949 - val_loss: 0.3661 - val_accuracy: 0.8242 - lr: 0.0010
Epoch 9/15
32/32 [==============================] - ETA: 0s - loss: 0.4139 - accuracy: 0.8008
Epoch 9: val_loss did not improve from 0.36607
32/32 [==============================] - 15s 452ms/step - loss: 0.4139 - accuracy: 0.8008 - val_loss: 0.4061 - val_accuracy: 0.8125 - lr: 0.0010
Epoch 10/15
32/32 [==============================] - ETA: 0s - loss: 0.4293 - accuracy: 0.7899
Epoch 10: val_loss did not improve from 0.36607
32/32 [==============================] - 10s 308ms/step - loss: 0.4293 - accuracy: 0.7899 - val_loss: 0.3975 - val_accuracy: 0.8359 - lr: 0.0010
Epoch 11/15
32/32 [==============================] - ETA: 0s - loss: 0.4079 - accuracy: 0.8067
Epoch 11: val_loss did not improve from 0.36607

Epoch 11: ReduceLROnPlateau reducing learning rate to 0.00010000000474974513.
32/32 [==============================] - 9s 294ms/step - loss:

INFO:tensorflow:Assets written to: E:\project\PDD\srp\assets


32/32 [==============================] - 44s 1s/step - loss: 0.3894 - accuracy: 0.8235 - val_loss: 0.3457 - val_accuracy: 0.8359 - lr: 1.0000e-04
Epoch 14/15
32/32 [==============================] - ETA: 0s - loss: 0.3761 - accuracy: 0.8274
Epoch 14: val_loss did not improve from 0.34569
32/32 [==============================] - 15s 455ms/step - loss: 0.3761 - accuracy: 0.8274 - val_loss: 0.3856 - val_accuracy: 0.8086 - lr: 1.0000e-04
Epoch 15/15
32/32 [==============================] - ETA: 0s - loss: 0.3816 - accuracy: 0.8333
Epoch 15: val_loss improved from 0.34569 to 0.34163, saving model to E:\project\PDD\srp
INFO:tensorflow:Assets written to: E:\project\PDD\srp\assets


INFO:tensorflow:Assets written to: E:\project\PDD\srp\assets


32/32 [==============================] - 54s 2s/step - loss: 0.3816 - accuracy: 0.8333 - val_loss: 0.3416 - val_accuracy: 0.8555 - lr: 1.0000e-04


In [13]:
# Evaluate the model
evaluation = model.evaluate(validation_generator)

# Print evaluation results
print("Validation Loss:", evaluation[0])
print("Validation Accuracy:", evaluation[1])


9/9 [==============================] - 3s 265ms/step - loss: 0.3413 - accuracy: 0.8731
Validation Loss: 0.3412567675113678
Validation Accuracy: 0.8730769157409668


In [14]:
# Evaluate the model
evaluation = model.evaluate(train_generator)

# Print evaluation results
print("Validation Loss:", evaluation[0])
print("Validation Accuracy:", evaluation[1])


33/33 [==============================] - 9s 280ms/step - loss: 0.3398 - accuracy: 0.8499
Validation Loss: 0.3398076891899109
Validation Accuracy: 0.8499044179916382


In [17]:
import cv2 as cv
import numpy as np
# image_path = "D:/srp_pdd/datasets/Spiral/test/Parkinson/Parkinson32.png"
image_path = "D:/srp_pdd/datasets/Spiral/test/Healthy/Healthy3.png"
img = cv.imread(image_path)
img = cv.resize(img,(100,100))
img = img/255.0
img = np.expand_dims(img,axis=0)
predictions = model.predict(img)

if predictions[0][0]>predictions[0][1]:
    print("class 0")
    print(list(train_generator.class_indices.keys())[0])
else:
    print("class 1")
    print(list(train_generator.class_indices.keys())[1])

1/1 [==============================] - 0s 90ms/step
class 0
Healthy
